In [1]:
import h5py
import ast
from utils.misc_utils import get_logger

import numpy as np
from scipy import stats
from scipy.special import gamma, digamma
from scipy.integrate import nquad

from unite_toolbox.bin_estimators import calc_bin_kld

In [2]:
from utils.bin_evaluators import EvaluatorBIN

eval = EvaluatorBIN()

eval.data_path = "data.hdf5"
eval.out_path = "results/bin.hdf5"
eval.logger = get_logger("results/bin_kld.log")

eval.quantity = "kld"

eval.hyper_params = ["scott", "fd", "sturges"]
eval.sample_sizes = [100, 200, 500, 1_000, 5_000, 10_000, 50_000, 100_000]
eval.seeds = range(1, 6)

for k, v in vars(eval).items():
    print(f"{k} - {v}")


eval.create_database()
eval.create_group()

experiments = [
    "uniform",
    "normal",
    "normal-mixture",
    "exponential",
    "bivariate-normal",
    "bivariate-normal-mixture",
    "gexp",
    "4d-gaussian"
]

data_path - data.hdf5
out_path - results/bin.hdf5
logger - <RootLogger root (DEBUG)>
quantity - kld
hyper_params - ['scott', 'fd', 'sturges']
sample_sizes - [100, 200, 500, 1000, 5000, 10000, 50000, 100000]
seeds - range(1, 6)
results - None


In [3]:
# # # # # UNIFORM # # # # #

experiment = "uniform"
# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

true_kld = np.log(dist2_params[0][1] / dist1_params[0][1])

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "uniform||uniform", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:32 - Creating converter from 3 to 5
2023-11-13 12:59:32 - (UNIFORM, scott, 10000, 1) - Time: 0.00091 s - Est.: 0.259 nats
2023-11-13 12:59:32 - (UNIFORM, scott, 10000, 2) - Time: 0.00098 s - Est.: 0.266 nats
2023-11-13 12:59:32 - (UNIFORM, scott, 10000, 3) - Time: 0.00066 s - Est.: 0.262 nats
2023-11-13 12:59:32 - (UNIFORM, scott, 10000, 4) - Time: 0.00110 s - Est.: 0.275 nats
2023-11-13 12:59:33 - (UNIFORM, scott, 10000, 5) - Time: 0.00065 s - Est.: 0.263 nats
2023-11-13 12:59:33 - Creating converter from 5 to 3


True entropy: 0.288 nats


In [4]:
# # # # # NORMAL # # # # #

experiment = "normal"
# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

true_kld = 0.5 * (
    (dist1_params[0][1]/dist2_params[0][1]) ** 2 +
    (dist2_params[0][0] - dist1_params[0][0]) ** 2 / (dist2_params[0][1] ** 2) -
    1 + np.log((dist2_params[0][1]**2)/(dist1_params[0][1]**2))
) # Reference

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "normal||normal", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:33 - (NORMAL, scott, 10000, 1) - Time: 0.00074 s - Est.: 0.362 nats
2023-11-13 12:59:33 - (NORMAL, scott, 10000, 2) - Time: 0.00068 s - Est.: 0.386 nats
2023-11-13 12:59:33 - (NORMAL, scott, 10000, 3) - Time: 0.00080 s - Est.: 0.377 nats
2023-11-13 12:59:33 - (NORMAL, scott, 10000, 4) - Time: 0.00094 s - Est.: 0.377 nats
2023-11-13 12:59:33 - (NORMAL, scott, 10000, 5) - Time: 0.00068 s - Est.: 0.360 nats


True entropy: 0.361 nats


In [5]:
# # # # # NORMAL-MIXTURE # # # # #

experiment = "normal-mixture"
# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

def pdf_normal(x, params):
    y = 0.0
    for dist in params:
        l, s, w = dist
        y += stats.norm(loc=l, scale=s).pdf(x) * w
    return y

def kld_normals(x, params1, params2):
    p = pdf_normal(x, params1)
    q = pdf_normal(x, params2)
    return p * np.log(p / q)

norm_lims = [[-15, 25]]

true_kld = nquad(kld_normals, norm_lims, args=(dist1_params, dist2_params,))[0] # Numerical Integration Solution


eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "normal-mixture||normal", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:33 - (NORMAL-MIXTURE, scott, 10000, 1) - Time: 0.00072 s - Est.: 0.181 nats
2023-11-13 12:59:33 - (NORMAL-MIXTURE, scott, 10000, 2) - Time: 0.00085 s - Est.: 0.178 nats
2023-11-13 12:59:34 - (NORMAL-MIXTURE, scott, 10000, 3) - Time: 0.00071 s - Est.: 0.178 nats
2023-11-13 12:59:34 - (NORMAL-MIXTURE, scott, 10000, 4) - Time: 0.00072 s - Est.: 0.182 nats
2023-11-13 12:59:34 - (NORMAL-MIXTURE, scott, 10000, 5) - Time: 0.00147 s - Est.: 0.171 nats


True entropy: 0.179 nats


In [6]:
# # # # # EXPONENTIAL # # # # #

experiment = "exponential"
# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

true_kld = np.log(1 / dist1_params[0][1]) - np.log(1 / dist2_params[0][1]) + dist1_params[0][1] / dist2_params[0][1] - 1 # Reference

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "exp||exp", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:34 - (EXPONENTIAL, scott, 10000, 1) - Time: 0.00090 s - Est.: 0.656 nats
2023-11-13 12:59:34 - (EXPONENTIAL, scott, 10000, 2) - Time: 0.00066 s - Est.: 0.628 nats
2023-11-13 12:59:34 - (EXPONENTIAL, scott, 10000, 3) - Time: 0.00055 s - Est.: 0.640 nats
2023-11-13 12:59:34 - (EXPONENTIAL, scott, 10000, 4) - Time: 0.00112 s - Est.: 0.605 nats
2023-11-13 12:59:34 - (EXPONENTIAL, scott, 10000, 5) - Time: 0.00057 s - Est.: 0.621 nats


True entropy: 0.636 nats


In [7]:
# # # # # BIVARIATE NORMAL # # # # #

experiment = "bivariate-normal"
# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

m1, s1, _ = dist1_params[0]
m2, s2, _ = dist2_params[0]
m1, s1, m2, s2 = [np.array(p) for p in [m1, s1, m2, s2]]

true_kld = 0.5 * (
    np.log(np.linalg.det(s2)/np.linalg.det(s1)) + 
    np.trace(np.linalg.inv(s2) @ s1) +
    (m2 - m1).T @ np.linalg.inv(s2) @ (m2 - m1) -
    len(m2)
)

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "exp||exp", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:34 - (BIVARIATE-NORMAL, scott, 10000, 1) - Time: 0.00156 s - Est.: 0.959 nats
2023-11-13 12:59:34 - (BIVARIATE-NORMAL, scott, 10000, 2) - Time: 0.00209 s - Est.: 1.003 nats
2023-11-13 12:59:34 - (BIVARIATE-NORMAL, scott, 10000, 3) - Time: 0.00142 s - Est.: 0.985 nats
2023-11-13 12:59:34 - (BIVARIATE-NORMAL, scott, 10000, 4) - Time: 0.00174 s - Est.: 0.978 nats
2023-11-13 12:59:34 - (BIVARIATE-NORMAL, scott, 10000, 5) - Time: 0.00140 s - Est.: 0.971 nats


True entropy: 0.949 nats


In [8]:
# # # # # BIVARIATE-NORMAL-MIXTURE # # # # #

experiment = "bivariate-normal-mixture"
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

def pdf_mnorm(x, y, params):
    z = 0.0
    for dist in params:
        l, s, w = dist
        z += stats.multivariate_normal(mean=l, cov=s).pdf(np.dstack((x, y))) * w
    return z

def kld_mnorms(x, y, params1, params2):
    p = pdf_mnorm(x, y, params1)
    q = pdf_mnorm(x, y, params2)
    return p * np.log(p / q)

mnorm_lims = [[-7, 7], [-7, 7]]

true_kld = nquad(kld_mnorms, mnorm_lims, args=(dist1_params, dist2_params,))[0]

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "bivariate-normal-mixture||bivariate-normal", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:42 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 1) - Time: 0.00173 s - Est.: 0.341 nats
2023-11-13 12:59:42 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 2) - Time: 0.00142 s - Est.: 0.391 nats
2023-11-13 12:59:42 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 3) - Time: 0.00139 s - Est.: 0.353 nats
2023-11-13 12:59:42 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 4) - Time: 0.00145 s - Est.: 0.364 nats
2023-11-13 12:59:42 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 5) - Time: 0.00160 s - Est.: 0.347 nats


True entropy: 0.312 nats


In [9]:
# # # # # GAMMA-EXPONENTIAL # # # # #

experiment = "gexp"

# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

def pdf_gamma_exponential(x, y, params):
    z = 0.0
    for dist in params:
        t, w = dist
        z += (1 / gamma(t)) * (x**t) * np.exp(-x - x * y) * w
    return z

def kld_gamma_exponentials(x, y, params1, params2):
    p = pdf_gamma_exponential(x, y, params1)
    q = pdf_gamma_exponential(x, y, params2)
    return p * np.log(p / q)

gexp_lims = [[0, 15], [0, 12]]

true_kld = nquad(kld_gamma_exponentials, gexp_lims, args=(dist1_params, dist2_params,))[0]

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "gexp||gexp", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:43 - (GEXP, scott, 10000, 1) - Time: 0.00162 s - Est.: 0.169 nats
2023-11-13 12:59:43 - (GEXP, scott, 10000, 2) - Time: 0.00188 s - Est.: 0.177 nats
2023-11-13 12:59:43 - (GEXP, scott, 10000, 3) - Time: 0.00208 s - Est.: 0.181 nats
2023-11-13 12:59:43 - (GEXP, scott, 10000, 4) - Time: 0.00201 s - Est.: 0.181 nats
2023-11-13 12:59:43 - (GEXP, scott, 10000, 5) - Time: 0.00208 s - Est.: 0.196 nats


True entropy: 0.175 nats


In [10]:
eval.seeds = range(1, 2)

# # # # # 4D-GAUSSIAN # # # # #

experiment = "4d-gaussian"

# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist1_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])
    dist2_params = ast.literal_eval(f[experiment]["q"].attrs["hyper_params"])

def kld_scipy_mnorm(d1, d2):
    a = np.log(np.linalg.det(d2.cov) / np.linalg.det(d1.cov))
    b = np.trace(np.linalg.inv(d2.cov) @ d1.cov)
    c = (d1.mean - d2.mean) @ np.linalg.inv(d2.cov) @ (d1.mean - d2.mean).T
    n = len(d1.mean)

    kld = 0.5 * (a + b) + 0.5 * (c - n)
    return kld

dist1 = stats.multivariate_normal(mean=dist1_params[0][0], cov=dist1_params[0][1])
dist2 = stats.multivariate_normal(mean=dist2_params[0][0], cov=dist2_params[0][1])

true_kld = kld_scipy_mnorm(dist1, dist2)

eval.evaluate_kld(experiment, "scott")

# Save
eval.write_double_to_hdf5(experiment, "4dgauss||4dgauss", true_kld)
print(f"True entropy: {true_kld:.3f} nats")

2023-11-13 12:59:45 - (4D-GAUSSIAN, scott, 10000, 1) - Time: 0.74588 s - Est.: 0.002 nats


True entropy: 0.901 nats
